In [8]:
from transformers import *
import torch as t
import os
import warnings

warnings.filterwarnings("ignore")

In [9]:
def get_latest_model_folder(model_dir, file_start):
    model_files = [file for file in os.listdir(model_dir) if file_start in file]
    model_files.sort()
    return os.path.join(model_dir, model_files[-1])


In [10]:
model_dir = 'saved_runs'
file_start = 'mod_digit_add_'
config = Config()
latest_model_folder = get_latest_model_folder(model_dir, file_start)
latest_model_folder

'saved_runs/mod_digit_add_2024-09-16_23-55'

In [15]:
num_models = 10 
num_epochs = 15000
models = [Transformer(config) for _ in range(num_models)]
weights = []#[t.load(latest_model_folder + '/init.pth')]
weights += [t.load(latest_model_folder + f'/{int((i*num_epochs)/(num_models))}.pth') for i in range(1,num_models - 1)] 
#weights += [t.load(latest_model_folder + '/final.pth')]

idx = []


for i in range(len(weights)):

    models[i].load_state_dict(weights[i]['model'])
    models[i].eval()

tokenizer = Tokenizer(config)

In [16]:
def get_accuracy(model):

    correct = 0
    count = 0
    for i in range(113):
        for j in range(113):
            count += 1
            print(f"Accuracy: {(correct/count):.2%}, Count: {count}", end = '\r')
            correct_answer = (i + j) % 113
            lsi = [int(k) for k in str(i)]
            lsj = [int(k) for k in str(j)]
            question = lsi + [10] + lsj + [11]

            ll = len(question)
            #print("q:", question)
            pred = model.generate_greedy(question)
            answer = pred[ll: -1]
            try:
                answer = int(tokenizer.detokenize(answer))
            except:
                continue
            #print("p", pred)
            #print("a", answer)

            if answer == correct_answer:
                correct += 1
            
            
    print('\n')

for i, model in enumerate(models):
    print(f"Calculating accuracy for model {i}")
    get_accuracy(model)

         

Calculating accuracy for model 0


In [40]:
test_sentence = "1+1="
test_sentence_tokenized = tokenizer.tokenize(test_sentence)

pred = model.generate_greedy([1, 10, 2,1, 11])
print("Prediction:", pred)

Prediction: [1, 10, 2, 1, 11, 2, 2, 12]


In [41]:
test_sentence_tokenized

[1, 10, 1, 11]